## 4 - Convolutional Sentiment Analysis

> Ở notebooks trước, chúng ta đã đạt được accuracy xấp xỉ 85% bằng cách sử dụng RNNs và áp dụng mô hình trong paper [Bag of Tricks for Efficient Text Classification](https://arxiv.org/pdf/1607.01759.pdf). Trong notebook này, chúng ta sẽ cùng nhau tìm hiểu và áp dụng *convolutional neural network (CNN) để xây dựng một mô hình phân tích cảm xúc, áp dụng mô hình trong bài báo [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/pdf/1408.5882.pdf)*.

> LƯU Ý: Notebook này chỉ tóm gọn về convolutional neural network. Để biết thêm chi tiết về CNNs, bạn có thể tham khảo [CS231n: Convolutional Neural Networks for Visual Recognition](http://cs231n.github.io/convolutional-networks/). và [Convolutional Neural Networks Coursera](https://www.coursera.org/learn/convolutional-neural-networks), hoặc [Convolutional Neural Networks blog](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/).

> Thông thường, CNNs được sử dụng trong phân tích ảnh. CNNs thường được tạo bởi một hoặc nhiều lớp tích chập - *convolutional* layers, theo sau là một hoặc nhiều *linear layers*. *Convolutional* layers sử dụng các *filters* (còn được gọi là *kernels* hoặc *receptive fields*) quét ngang qua một bức ảnh và sinh ra một bức ảnh mới đã được xử lý. Bức ảnh mới này có thể đưa qua các lớp covolutional hoặc linear tùy bài toán. Mỗi filter đều có kích thước, ví dụ một filter 3x3 có chiều dài là 3, rộng 3 và tổng cộng có 9 trọng số. Trong xử lý ảnh truyền thống, các trọng số này sẽ được tính toán bằng tay. Tuy nhiên, với CNNs, các trọng số này được học thông qua *backpropagation*.

> Ý tưởng của việc học các trọng số là mỗi *convolutional* layer hoạt động như một bộ trích xuất đặc trưng - *feature extractors*, trích xuất những phần quan trọng trong bức ảnh. Ví dụ, nếu sử dụng CNN để phát hiện khuôn mặt, CNN sẽ nhìn vào những đặc trưng quan trọng của khuôn mặt như: mắt, mũi, miệng, lông mày, ....

> Thế, tại sao lại sử dụng CNNs cho văn bản? Tương tự như việc xử lý ảnh với filter 3x3, một filter 1x2 có thể nhanh chóng kiểm tra 2 từ liền nhau trong một đoạn văn bản, ví dụ: bi-gram. Ở Notebook trước, chúng ta sử dụng FastText model - sử dụng bi-grams bằng cách chèn chúng vào cuối mỗi câu. Lần này, chúng ta sẽ sử dụng CNNs để học bi-grams (1x2 filter) một cách tự động, bên cạnh đó còn học tri-grams (1x3 filter), ..., n-grams (1xn filter) trong văn bản. 

> Trực giác cho thấy rằng: nếu có một số bi-grams, tri-grams và n-grams nhất định xuất hiện trong bài đánh giá, đó sẽ là dấu hiệu tốt để đưa ra kết luận cuối cùng.

### Preparing Data

> Ta xây dựng tập dữ liệu tương tự notebook 2. <br>
> Do convolutional layers yêu cầu số chiều của batch phải nằm trước nên ta sử dụng `batch_first=True` khi khởi tạo `DataLoader`.

In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                    tokenizer_language='en_core_web_sm',
                    batch_first = True)

LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

> Xây dựng vocab và load pre-trained word embeddings.

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                    max_size = MAX_VOCAB_SIZE,
                    vectors = "glove.6B.100d",
                    unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

> Tương tự như các notebook trước, ta tạo iterators.

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = \
    data.BucketIterator.splits(
        (train_data, valid_data, test_data),
        batch_size = BATCH_SIZE,
        device = device) 

### Build the Model